In [11]:
import numpy as np 
import pandas as pd
import requests
import datetime as dt
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import pickle

class SVMPredictor :
    def __init__(self,symbol="BTCUSDT",interval="1M",api="https://api.binance.com/api/v3/klines"):
        self.options = {
            "symbol":symbol,
            "interval":interval,
            "limit":"1000"
        }
        self.api = api
        self.prediction_time_space = 5

    def fetchJSON(self):
        return requests.get(url=self.api,params=self.options).json()

    def createDF(self):
        self.DF = pd.DataFrame(self.fetchJSON())
        self.DF.columns = ['open_time','o', 'h', 'l', 'c', 'v','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
        self.latest_timestamp = list(self.DF.close_time)[-1:][0]
        
    def filterDF(self):
        self.df = pd.DataFrame({"c":self.DF["c"].astype('float64')})

    def initDataset(self,x,y):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size=0.2)

    def initSVR(self,kernel="rbf",C=1e5,gamma=0.000001):
        self.svr = SVR(kernel=kernel,C=C,gamma = gamma)

    def trainSVR(self):
        self.svr.fit(self.x_train,self.y_train)
    
    def getScore(self):
        return self.svr.score(self.x_test,self.y_test)

    def runSVR(self):
        x = np.array(self.df["c"])[:-self.prediction_time_space].reshape(-1, 1)
        y = np.array(self.df["c"])[self.prediction_time_space:]
        self.initDataset(x,y)
        self.initSVR()
        self.trainSVR()

    def dumpModel(self):
        pickle.dump(self.svr, open("svr_1", 'wb'))
    
    def loadModel(self):
        self.svr = pickle.load(open("svr_1", "rb"))
    
    def runForecast(self):
        self.forecasted_data = self.svr.predict(np.array(self.df["c"])[-self.prediction_time_space:].reshape(-1, 1))
    
    def runPrediction(self):
        self.prediction = self.svr.predict(np.array(self.df["c"])[:-self.prediction_time_space].reshape(-1, 1))

    def plotPrediction(self):
        pd.DataFrame({"Prediction":self.prediction}).plot()
    
    def plotForecast(self):
        pd.DataFrame({"Forecast":self.forecasted_data}).plot()
    
predictor = SVMPredictor()
pickle.dump(predictor.fetchJSON()["c"],open("btc_data","wb"))
#print(pickle.load(open("btc_data","rb")))
'''predictor.createDF()
predictor.filterDF()
predictor.runSVR()
predictor.runForecast()
predictor.plotForecast()'''


TypeError: list indices must be integers or slices, not str

In [1]:
import numpy as np 
import pandas as pd
import requests
import datetime as dt
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import pickle
from prophet import Prophet
#from fbprophet import Prophet

df = pd.read_csv("btc.csv")
df["Timestamp"] =  pd.to_datetime(df["Timestamp"],unit='s')
#df = df.groupby([pd.Grouper(key='Timestamp', freq='H')]).first().reset_index()
#df = df.set_index('Timestamp')
df = df.set_index('Timestamp')
df = df.resample('1h').mean()
df = df[['Weighted_Price']]
df['Weighted_Price'].fillna(method='ffill', inplace=True)
df = df.reset_index().rename(columns = {'Timestamp':'ds','Weighted_Price':'y'})
df = df.set_index('ds')
df.plot(y=["y"],figsize=(20,5))


ModuleNotFoundError: No module named 'prophet'

In [8]:

import numpy as np
import pandas as pd
import requests
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import uvicorn
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from typing import Optional
options = {
        "symbol": 'BTCUSDT',
        "interval": '5m',
        "limit": "1000"
    }

    #fetch btc price data in json
x = requests.get(
        url="https://api.binance.com/api/v3/klines", params=options)
df = pd.DataFrame(x.json())
df.columns = ['open_time', 'o', 'h', 'l', 'c', 'v', 'close_time',
                  'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore']
df.index = df.close_time
df.drop(columns=['open_time', 'o', 'h', 'l', 'v', 'close_time', 'qav',
            'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore'], inplace=True)
df["Prediction"] = df["c"].shift(-7)
print(df)

                            c      Prediction
close_time                                   
1620857399999  53021.94000000  52317.45000000
1620857699999  52608.36000000  52533.43000000
1620857999999  52837.01000000  52536.89000000
1620858299999  53072.10000000  52942.79000000
1620858599999  52748.44000000  52415.09000000
...                       ...             ...
1621155899999  48998.45000000             NaN
1621156199999  49105.74000000             NaN
1621156499999  49061.25000000             NaN
1621156799999  49176.39000000             NaN
1621157099999  49044.99000000             NaN

[1000 rows x 2 columns]
